In [1]:
%%writefile requirements.txt
streamlit>=1.26.0
streamlit-feedback
pypdf
pyngrok

Writing requirements.txt


In [2]:
!pip install -r requirements.txt --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 42.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 49.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.7 MB/s eta 0:00:00


In [19]:
%%writefile ER_analysis_app.py

import streamlit as st
import pypdf
import json
import requests
from google.colab import userdata
from pyngrok import ngrok

# Function to read PDF and extract text
def read_pdf(file):
	  pages = []
	  pdf = pypdf.PdfReader(file)
	  for p in range(len(pdf.pages)):
		  page = pdf.pages[p]
		  text = page.extract_text()
		  pages += [text]
	  return pages

# Set layout
st.set_page_config(page_title="Earning Release Analyst", layout="wide")

# Set Title
st.title("📝 Earning Release Analyst")

# Set Sidebar
with st.sidebar:
  hf_api_key = st.text_input("Enter your Hugging Face API key", type="password")
  ngrok_auth_token = st.text_input("Enter your NGROK token", type="password")

# Upload the file
uploaded_file = st.file_uploader("Upload a file", type=("txt", "md", "pdf"))


if uploaded_file and hf_api_key and ngrok_auth_token:

    # Parse the file
    if uploaded_file.type == "application/pdf":
        article = read_pdf(uploaded_file)
    else:
        article = uploaded_file.read().decode()

    # Prompt for LLM
    prompt = f"""
    Act as a company financial analyst to analyze the earnings releases of companies,
    delimited below by triple backticks.

    Output should include:

    - Overall sentiment: <positive, negative or neutral; keeping in mind that companies will often appear to be extremely optimistic about their company performance>
    - Key words: <find those words which are most repeated and drivers of growth and performance for the company, include 7 words max>
    - Executive summary: <executive summary of past performance and about which will be drivers for next earnings in the future, 10 lines max>

    Keep all information factual and only include information extracted from the earning release document.

    Earning release document:
    ```{article}```
    """

    # Define query function
    def query(payload, model_api_url):
        headers = {"Authorization": f"Bearer {hf_api_key}"}
        response = requests.post(model_api_url, headers=headers, json=payload)
        return response.json()

    # LLM model parameters
    mistral_api_url = "https://api-inference.huggingface.co/models/mistralai/Mistral-7B-Instruct-v0.2"
    mistral_params = {
                  "wait_for_model": True,
                  "do_sample": False,
                  "return_full_text": False,
                  "max_new_tokens": 5000,
                  "repetition_penalty": 1.5,
                  "temperature": 0.001
                }

    # LLM model request
    output = query(payload={"inputs": prompt, "parameters": mistral_params}, model_api_url=mistral_api_url)

    # Display result
    st.write("### Answer")
    st.write(output[0]['generated_text'])

Overwriting ER_analysis_app.py


In [20]:
!streamlit run /content/ER_analysis_app.py &>/content/logs.txt &

In [21]:
# Setup ngrok
from pyngrok import ngrok
from google.colab import userdata

# Terminate open tunnels if exist
ngrok.kill()

# Setting the authtoken using Google Secrets
ngrok_auth_token = userdata.get('NGROK_KEY')
ngrok.set_auth_token(ngrok_auth_token)

# Open an HTTPs tunnel on port XXXX which you get from your `logs.txt` file
ngrok_tunnel = ngrok.connect(addr="8505", proto="http", bind_tls=True)
print("Streamlit App:", ngrok_tunnel.public_url)

Streamlit App: https://84a3-35-194-79-254.ngrok-free.app


In [18]:
ngrok.kill()